In [3]:
import cv2
import numpy as np

# Load the image
image_path = 'Fungal Area/13.jpg'
image = cv2.imread(image_path)

# Resize the image
scale_percent = 20
width = int(image.shape[1] * scale_percent / 100)
height = int(image.shape[0] * scale_percent / 100)
resized_image = cv2.resize(image, (width, height))

# Define the cropping coordinates
top_left_corner = (46, 68)
bottom_right_corner = (392, 578)


# Crop the image using array slicing
cropped_image = resized_image[top_left_corner[1]:bottom_right_corner[1], top_left_corner[0]:bottom_right_corner[0]]

# Apply median filtering
kernel_size = 3
median_filtered_image = cv2.medianBlur(cropped_image, kernel_size)

# Sharpen the image using the Laplacian on the original color image
laplacian = cv2.Laplacian(median_filtered_image, cv2.CV_64F)
laplacian_display = cv2.convertScaleAbs(laplacian)
sharpened_image = cv2.convertScaleAbs(median_filtered_image - laplacian)

# Convert the sharpened image to HSV color space
hsv_image = cv2.cvtColor(sharpened_image, cv2.COLOR_BGR2HSV)

# Define range for yellow color and create mask
lower_yellow = np.array([22, 100, 100])
upper_yellow = np.array([30, 255, 255])
yellow_mask = cv2.inRange(hsv_image, lower_yellow, upper_yellow)

# Perform morphological operations to clean up the yellow mask
kernel = np.ones((3, 3), np.uint8)
yellow_mask = cv2.morphologyEx(yellow_mask, cv2.MORPH_CLOSE, kernel, iterations=2)
yellow_mask = cv2.morphologyEx(yellow_mask, cv2.MORPH_OPEN, kernel, iterations=2)

# Find contours on the yellow mask
yellow_contours, _ = cv2.findContours(yellow_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
segmented_yellow_image = sharpened_image.copy()
cv2.drawContours(segmented_yellow_image, yellow_contours, -1, (0, 255, 255), 2)

# Calculate area of yellow colonies
yellow_area = sum(cv2.contourArea(contour) for contour in yellow_contours)

# Display the results
cv2.imshow('Resized Image', resized_image)
cv2.imshow('Cropped Image', cropped_image)
cv2.imshow('Median Filtered Image', median_filtered_image)
cv2.imshow('Laplacian Image', laplacian_display)
cv2.imshow('Sharpened Image', sharpened_image)
cv2.imshow('Yellow Colony Segmentation', yellow_mask)
cv2.imshow('Segmented Yellow Image', segmented_yellow_image)

print(f"Total area of yellow colonies: {yellow_area}")

cv2.waitKey(0)
cv2.destroyAllWindows()


Total area of yellow colonies: 43969.0
